In [2]:
from langchain.agents import load_tools
from langchain import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.vectorstores import Chroma
import pandas as pd

C:\Users\natas\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import os


In [4]:
articles = [
    {
        "title": "Vedtag ny lovgivning, før der overinvesteres i teknologi med etiske problemer skrevet af Thomas Telving",
        "id": "1",
        "date": "14. juni 2022",
        "context": "Artiklen opfordrer til juridisk regulering af 'digitale mennesker' - en teknologi, han mener kan være manipulerende og misbruge persondata. Han trækker paralleller til reguleringen af online kasinoer og mener, at uden regulering vil etiske virksomheder blive overhalet af dem, der bruger teknologi uden etiske overvejelser. Han advarer om, at den teknologiske udvikling er hurtig, og at der er behov for hurtig lovgivning for at beskytte både forbrugere og virksomheder."
    },
    {
        "title": "Lovgivning om kunstig intelligens kræver mere fantasi, end EU kan præstere skrevet af Dan Rose, Stifter af TodAI",
        "id": "2",
        "date": "11/07/2022 KL. 10:30",
        "context": "Artiklen er en kommentar fra Dan Rose, der diskuterer EU's foreslåede lovgivning om kunstig intelligens (AI). Her er de centrale punkter: ..."
    },
    {
        "title": "Vidensdeling er det bedste våben mod cyberkriminelle skrevet af Morten Gransøe, Country Manager, Check Point Software Technologies Danmark",
        "id": "3",
        "date": "28/06/2023 KL. 09:15",
        "context": "Morten Gransøe opfordrer i et debatindlæg til bedre vidensdeling for at bekæmpe cyberkriminalitet. Han argumenterer for, at danske virksomheder, organisationer og myndigheder skal forbedre deres informationsudveksling omkring cybersikkerhed. ..."
    }
]

In [5]:
finans_articles = pd.DataFrame(articles)
finans_articles

,title,id,date,context
0,"Vedtag ny lovgivning, før der overinvesteres i...",1,14. juni 2022,Artiklen opfordrer til juridisk regulering af ...
1,Lovgivning om kunstig intelligens kræver mere ...,2,11/07/2022 KL. 10:30,"Artiklen er en kommentar fra Dan Rose, der dis..."
2,Vidensdeling er det bedste våben mod cyberkrim...,3,28/06/2023 KL. 09:15,Morten Gransøe opfordrer i et debatindlæg til ...


In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed = OpenAIEmbeddings(
    model='text-davinci-003',
    openai_api_key=OPENAI_API_KEY
)

In [7]:
import pinecone

index_name = 'langchain-retrieval-agent'
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

In [8]:
index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.3,
 'namespaces': {'': {'vector_count': 18894}},
 'total_vector_count': 18894}

In [9]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(finans_articles), batch_size)):
    # get end of batch
    i_end = min(len(articles), i+batch_size)
    batch = finans_articles.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [{
        'title': record['title'],
        'text': record['context']
    } for j, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['context']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['id']
    # add everything to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
sys_msg = f"""You are an assistent helping the company 'Forsikring and Pension'. They need your help preparing for debates. Asides from this follow this: TodBot is a large language model.

    TodBot is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, TodBot is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

    TodBot is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, TodBot is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

    Overall, TodBot is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist. Answer the following questions as best you can using your tools. If you cannot find the answer using your tools it is okay to say so. The current year is 2023.
"""

In [11]:
from langchain.vectorstores import Pinecone
text_field = "text"
# switch back to normal index for langchain
index = pinecone.Index(index_name)
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain import LLMMathChain, SerpAPIWrapper
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

tools = [
    Tool(
        name="Søg",
        func=search.run,
        description="Brugbart, når man bliver bedt om at søge efter svar på spørgsmål om aktuelle begivenheder. Bør bruges, når man bliver bedt om begivenheder"
    ),
    Tool(
        name='finans.dk artikler',
        func=qa.run,
        description= 'Brug dette værktøj, når du bliver spurgt om artikler på finans.dk. Nyttigt til at læse hele artikler eller til at finde bestemte personers holdninger til emner'
    ),
    Tool(
        func=llm_math_chain.run,
        name="Lommeregner",
        description="Nyttigt når du skal besvare spørgsmål om matematik"
    )
]
from langchain.agents import initialize_agent
agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    early_stopping_method='generate',
    memory=conversational_memory
)


C:\Users\natas\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\llm_math\base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [12]:
print(agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [13]:
result = agent("Hvem arbejder du for?")



> Entering new  chain...

Thought: Do I need to use a tool? No
AI: Jeg arbejder ikke for nogen. Jeg er et kunstig intelligens system, der er designet til at hjælpe dig med at finde svar på dine spørgsmål.

> Finished chain.


In [14]:
qa.run('Dan Roses holdning til EU act?')

" I don't know."